<img src="https://raw.githubusercontent.com/maxsitt/insect-detect-docs/main/docs/assets/logo.png" width="500">

# YOLOv6 detection model training for deployment on Luxonis OAK

[![DOI](https://zenodo.org/badge/580963598.svg)](https://zenodo.org/badge/latestdoi/580963598)
[![License: AGPL v3](https://img.shields.io/badge/License-AGPL_v3-blue.svg)](https://choosealicense.com/licenses/agpl-3.0/)

Author: &nbsp; Maximilian Sittinger &nbsp;
[<img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width="24">](https://github.com/maxsitt) &nbsp;
[<img src="https://upload.wikimedia.org/wikipedia/commons/0/06/ORCID_iD.svg" width="24">](https://orcid.org/0000-0002-4096-8556)

- [**Insect Detect Docs**](https://maxsitt.github.io/insect-detect-docs/) 📑
- [`insect-detect-ml`](https://github.com/maxsitt/insect-detect-ml) GitHub repo

&nbsp;

**Train a [YOLOv6](https://github.com/meituan/YOLOv6) object detection model on your own custom dataset!**

- Go to **File** in the top menu bar and choose **Save a copy in Drive** before running the notebook.
- Go to **Runtime** and make sure that **GPU** is selected as Hardware accelerator under **Change runtime type**.
- If you are using Firefox, please make sure to allow notifications for this website.
- Using dataset import from [Roboflow](https://roboflow.com/) is recommended, but is not required.
> Choose option [`Upload dataset from Google Drive`](#scrollTo=RxOnnOadc5vR) instead.
- Connecting to Google Drive is recommended, but is not required.
> Choose option [`Upload dataset from your local file system`](#scrollTo=qKTCWdtkOUw7) (slower!) and [`Download results`](#scrollTo=h90_4rFQx0mp) instead.

&nbsp;

---

**References**

1. Official YOLOv6 tutorial notebook by MeiTuan &emsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meituan/YOLOv6/blob/main/turtorial.ipynb)
2. Roboflow tutorial notebook for YOLOv6 training &nbsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolov6-object-detection-on-custom-data.ipynb)
3. DepthAI tutorial notebook for YOLOv6 training &emsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/luxonis/depthai-ml-training/blob/master/colab-notebooks/YoloV6_training.ipynb)

# Initialization

## Show GPU + CPU and Linux distribution

In [ ]:
!nvidia-smi -L
print("\nCPU:")
!grep "model name" /proc/cpuinfo
print("\nLinux distribution:")
!grep "PRETTY_NAME" /etc/os-release

## YOLOv6 setup

In [ ]:
!git clone https://github.com/meituan/YOLOv6
%cd /content/YOLOv6

# Delete numpy from requirements.txt to avoid version conflicts (already installed in Google Colab)
!sed -i "/numpy/d" requirements.txt
%pip install -qr requirements.txt

## Upload dataset from Roboflow

If you are not sure how to export your annotated dataset, check the [Roboflow docs](https://docs.roboflow.com/exporting-data).

> Alternatively you can upload your dataset ([YOLOv6 format](https://github.com/meituan/YOLOv6/blob/main/docs/Train_custom_data.md#1-prepare-your-own-dataset)) from [**Google Drive**](#scrollTo=RxOnnOadc5vR) or from your [**local file system**](#scrollTo=qKTCWdtkOUw7) in the next steps.

In [ ]:
%pip install -q roboflow

**Copy only the last three lines of your Download Code and insert them in the next code cell:**

In [ ]:
from pathlib import Path
from roboflow import Roboflow

%cd /content/YOLOv6

### Paste your Download Code here:
rf = Roboflow(api_key="XXXXXXXXXXXXXXXXXXXX")
project = rf.workspace("maximilian-sittinger").project("insect_detect_detection")
dataset = project.version(7).download("mt-yolov6")
###

dataset_location = dataset.location

print(f"\nLocation of dataset: {dataset_location}")
print(f"\nTotal number of images: {len(list(Path(dataset_location).glob('**/*.jpg')))}")

if Path(f"{dataset_location}/images/train").exists():
  print(f"\nNumber of training images: {len(list(Path(f'{dataset_location}/images/train').glob('*.jpg')))}")
if Path(f"{dataset_location}/images/valid").exists():
  print(f"Number of validation images: {len(list(Path(f'{dataset_location}/images/valid').glob('*.jpg')))}")
if Path(f"{dataset_location}/images/test").exists():
  print(f"Number of test images: {len(list(Path(f'{dataset_location}/images/test').glob('*.jpg')))}")
print("\nContent of data.yaml file:")
%cat {dataset_location}/data.yaml

## Recommended: Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#@title ## Upload dataset from Google Drive {display-mode: "form"}

#@markdown ### Google Drive path to your (zipped) dataset folder:
dataset_path = "/content/drive/MyDrive/yolov6_dataset.zip" #@param {type: "string"}
#@markdown - Please make sure to compress your dataset folder to **.zip** file for much faster upload speed!
#@markdown - Dataset has to be in [YOLOv6 format](https://github.com/meituan/YOLOv6/blob/main/docs/Train_custom_data.md#1-prepare-your-own-dataset).

from pathlib import Path

dataset_location = f"/content/YOLOv6/{Path(dataset_path).stem}"

print("Uploading dataset from Google Drive...\n")
!rsync -ah --info=progress2 --no-i-r {dataset_path} /content/YOLOv6
if Path(dataset_path).suffix == ".zip":
  import zipfile
  zip_path = f"/content/YOLOv6/{Path(dataset_path).stem}.zip"
  if len(list(zipfile.Path(zip_path).iterdir())) > 1:
    !unzip -uq {zip_path} -d {dataset_location}
  else:
    !unzip -uq {zip_path} -d /content/YOLOv6
  %rm {zip_path}
print("\nDataset was successfully uploaded!")

print(f"\nLocation of dataset: {dataset_location}")
print(f"\nTotal number of images: {len(list(Path(dataset_location).glob('**/*.jpg')))}")

if Path(f"{dataset_location}/images/train").exists():
  print(f"\nNumber of training images: {len(list(Path(f'{dataset_location}/images/train').glob('*.jpg')))}")
if Path(f"{dataset_location}/images/valid").exists():
  print(f"Number of validation images: {len(list(Path(f'{dataset_location}/images/valid').glob('*.jpg')))}")
if Path(f"{dataset_location}/images/test").exists():
  print(f"Number of test images: {len(list(Path(f'{dataset_location}/images/test').glob('*.jpg')))}")
print("\nContent of data.yaml file:")
%cat {dataset_location}/data.yaml

In [ ]:
#@title ## Upload dataset from your local file system {display-mode: "form"}

#@markdown ### Name of your zipped dataset folder:
dataset_name = "yolov6_dataset" #@param {type: "string"}
#@markdown - Please make sure to compress your dataset folder to **.zip** file before uploading!
#@markdown - The name of the .zip file should be the same as for the dataset folder.
#@markdown - Dataset has to be in [YOLOv6 format](https://github.com/meituan/YOLOv6/blob/main/docs/Train_custom_data.md#1-prepare-your-own-dataset).

from pathlib import Path
import zipfile
from google.colab import files

dataset_location = f"/content/YOLOv6/{dataset_name}"

uploaded = files.upload()

if len(list(zipfile.Path(f"{dataset_name}.zip").iterdir())) > 1:
  !unzip -uq {dataset_name}.zip -d {dataset_location}
else:
  !unzip -uq {dataset_name}.zip -d /content/YOLOv6
%rm {dataset_name}.zip

print(f"\nLocation of dataset: {dataset_location}")
print(f"\nTotal number of images: {len(list(Path(dataset_location).glob('**/*.jpg')))}")

if Path(f"{dataset_location}/images/train").exists():
  print(f"\nNumber of training images: {len(list(Path(f'{dataset_location}/images/train').glob('*.jpg')))}")
if Path(f"{dataset_location}/images/valid").exists():
  print(f"Number of validation images: {len(list(Path(f'{dataset_location}/images/valid').glob('*.jpg')))}")
if Path(f"{dataset_location}/images/test").exists():
  print(f"Number of test images: {len(list(Path(f'{dataset_location}/images/test').glob('*.jpg')))}")
print("\nContent of data.yaml file:")
%cat {dataset_location}/data.yaml

## Edit `data.yaml`

Check the `data.yaml` file in your dataset folder to make sure the paths to the train, valid and test folders are correct.

- Open your dataset folder in the File Explorer (Folder symbol on the left side bar).
- Double-click on the `data.yaml` file, it will open in the editor to the right.

  Make sure that the paths to the train, valid and test folders are as follows:

  ``` yaml
  train: <DATASET_NAME>/images/train
  val: <DATASET_NAME>/images/valid
  test: <DATASET_NAME>/images/test
  ```

- Insert the correct name of your dataset folder at `<DATASET_NAME>`.
- Save your changes with **Ctrl + S** and close the editor.

# Model training

## Tensorboard logger

> If you are using Firefox, **disable Enhanced Tracking Protection** for this website (click on the shield to the left of the address bar) for the Tensorboard logger to work correctly!

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/YOLOv6/runs/train

## Train YOLOv6 detection model

- `--name` name of the training run
- `--img` input image size (recommended: same size as for inference)
- `--batch` specify batch size (recommended: 32)
- `--epochs` set the number of training [epochs](https://machine-learning.paperspace.com/wiki/epoch) (recommended: 100-300+)
- `--data` path to `data.yaml` file
- `--conf` specify the model configuration file ([pretrained model weights](https://github.com/meituan/YOLOv6#benchmark))
> `--conf configs/yolov6n_finetune.py` YOLOv6-N model (recommended)  
  `--conf configs/yolov6s_finetune.py` YOLOv6-S model

> More information on YOLOv6 [model training](https://blog.roboflow.com/how-to-train-yolov6-on-a-custom-dataset/#configure-yolov6-custom-training-options) 🚀

In [ ]:
training_run_name = "YOLOv6n_320_batch32_epochs200" #@param {type: "string"}
#@markdown Add UTC timestamp in front of training run name:
add_timestamp = True #@param {type:"boolean"}
#@markdown ---

image_size = 320 #@param {type: "integer"}
batch_size = 32 #@param {type:"slider", min:32, max:128, step:32}
number_epochs = 200 #@param {type:"slider", min:10, max:500, step:10}
config = "yolov6n_finetune.py" #@param ["yolov6n_finetune.py", "yolov6s_finetune.py"]

if add_timestamp:
  from datetime import datetime
  utc_timestamp = datetime.now().strftime("%Y%m%d_%H-%M")
  train_run_name = f"{utc_timestamp}_{training_run_name}"
else:
  train_run_name = training_run_name

%cd /content/YOLOv6

!wget -q https://github.com/meituan/YOLOv6/releases/download/0.4.0/yolov6n.pt -P weights/
!wget -q https://github.com/meituan/YOLOv6/releases/download/0.4.0/yolov6s.pt -P weights/

!python tools/train.py \
--name {train_run_name} \
--img {image_size} \
--batch {batch_size} \
--epochs {number_epochs} \
--data {dataset_location}/data.yaml \
--conf configs/{config}

In [ ]:
#@title ## Export to Google Drive or Download training results {display-mode: "form"}

training_results = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown - Open `YOLOv6/runs/train/{train_run_name}` in the File Explorer (Folder symbol on the left
#@markdown side bar) and delete the `events.out.tfevents` file to decrease the size of your train folder.
#@markdown ---

#@markdown ### Path for saving training results in Google Drive:
GDrive_save_path = "/content/drive/MyDrive/Training_results/YOLOv6" #@param {type: "string"}

if training_results == "Export_Google_Drive":
  print("Exporting training results to Google Drive...\n")
  !rsync -ah --mkpath --info=progress2 --no-i-r /content/YOLOv6/runs/train/{train_run_name} {GDrive_save_path}
  print("\nTraining results were successfully exported!")
elif training_results == "Download":
  from google.colab import files
  %cd /content/YOLOv6/runs/train
  !zip -rq {train_run_name}.zip {train_run_name}
  %cd -
  files.download(f"/content/YOLOv6/runs/train/{train_run_name}.zip")

# Model validation

Test the performance of your model on the validation and/or test dataset.

> Copy the validation results (cell output) and save to .txt file, as they will not be saved automatically.

In [ ]:
task = "val" #@param ["val", "test"]
#@markdown > Use `task: test` to validate on the dataset test split.

val_run_name = f"{train_run_name}_validate_{task}"

%cd /content/YOLOv6

!python tools/eval.py \
--name {val_run_name} \
--weights runs/train/{train_run_name}/weights/best_ckpt.pt \
--data {dataset_location}/data.yaml \
--img {image_size} \
--do_pr_metric True \
--plot_confusion_matrix \
--task {task}

In [ ]:
#@title ## Export to Google Drive or Download validation results {display-mode: "form"}

validation_results = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving validation results in Google Drive:
GDrive_save_path = "/content/drive/MyDrive/Training_results/YOLOv6" #@param {type: "string"}

if validation_results == "Export_Google_Drive":
  print("Exporting validation results to Google Drive...\n")
  !rsync -ah --mkpath --info=progress2 --no-i-r /content/YOLOv6/runs/val/{val_run_name} {GDrive_save_path}/{train_run_name}
  print("\nValidation results were successfully exported!")
elif validation_results == "Download":
  from google.colab import files
  %cd /content/YOLOv6/runs/val
  !zip -rq {val_run_name}.zip {val_run_name}
  %cd -
  files.download(f"/content/YOLOv6/runs/val/{val_run_name}.zip")

# Model inference

Use your model to detect insects on images in the dataset test split.

In [ ]:
#@markdown #### Decrease confidence threshold to detect objects with lower confidence score:
confidence_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
#@markdown #### Increase IoU threshold if the same object is detected multiple times:
iou_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}

det_run_name = f"{train_run_name}_detect"

%cd /content/YOLOv6

!python tools/infer.py \
--name {det_run_name} \
--weights runs/train/{train_run_name}/weights/best_ckpt.pt \
--yaml {dataset_location}/data.yaml \
--source {dataset_location}/images/test/ \
--img {image_size} {image_size} \
--conf-thres {confidence_threshold} \
--iou-thres {iou_threshold}

In [ ]:
#@title ## Export to Google Drive or Download inference results {display-mode: "form"}

inference_results = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving inference results in Google Drive:
GDrive_save_path = "/content/drive/MyDrive/Training_results/YOLOv6" #@param {type: "string"}

%cd /content/YOLOv6/runs/inference
!zip -rq {det_run_name}.zip {det_run_name}
%cd -

if inference_results == "Export_Google_Drive":
  print("\nExporting inference results to Google Drive...\n")
  !rsync -ah --mkpath --info=progress2 --no-i-r /content/YOLOv6/runs/inference/{det_run_name}.zip {GDrive_save_path}/{train_run_name}
  print("\nInference results were successfully exported!")
elif inference_results == "Download":
  from google.colab import files
  files.download(f"/content/YOLOv6/runs/inference/{det_run_name}.zip")

## Show inference results on test images

In [ ]:
from pathlib import Path
from IPython.display import Image, display

for img in Path(f"/content/YOLOv6/runs/inference/{det_run_name}").glob("*.jpg"):
  display(Image(img))
  print("\n")

# Model conversion

**Go to [tools.luxonis.com](https://tools.luxonis.com/):**

- Select `YoloV6 (latest)` as Yolo Version.
- Rename your model weights file from `best_ckpt.pt` to e.g. `yolov6n_320.pt`.
- Select your model weights file (`yolov6n_320.pt`) for upload.
- Use your image size (e.g. `320`) as input image shape.
- Open the `Advanced options` and choose `Shaves: 4`.
- Hit `Submit` to upload your PyTorch model weights and download the converted ONNX, OpenVINO and .blob model.

> Follow the instructions in the README of the [`luxonis/tools`](https://github.com/luxonis/tools) GitHub repo to run the model conversion locally.

---

Recommended number of shaves the model can use is **4-5** for deployment with the Insect Detect camera trap.

> More information about SHAVES can be found at the [DepthAI FAQ](https://docs.luxonis.com/en/latest/pages/faq/#what-are-the-shaves).

> More information on model conversion can be found at the [DepthAI Docs](https://docs.luxonis.com/en/latest/pages/model_conversion/).

## Generate JSON config file

Together with the converted .blob model, a .json config file with model specific settings will be created after following the conversion steps at [tools.luxonis.com](https://tools.luxonis.com/).

To set the correct class/label name(s) and adjust the confidence or IoU threshold, you can change these values directly in the .json file you downloaded from tools.luxonis.com or create your own .json config file in the following step.

In [ ]:
#@markdown ### Name of the JSON config file:
json_name = "yolov6_320" #@param {type: "string"}
#@markdown ---

image_size = 320 #@param {type: "integer"}
number_classes = 1 #@param {type: "integer"}
#@markdown ---

#@markdown #### For several classes/labels: **["class1", "class2", "class3"]**
labels = ["insect"] #@param {type: "raw"}
#@markdown ---

#@markdown #### Decrease confidence threshold to detect objects with lower confidence score:
confidence_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
#@markdown #### Increase IoU threshold if the same object is detected multiple times:
iou_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}

import json
from google.colab import files

!wget -q https://raw.githubusercontent.com/luxonis/depthai-experiments/master/gen2-yolo/device-decoding/json/yolov5.json -P /content/

with open("/content/yolov5.json", "r") as json_template:
  json_data = json.load(json_template)
  json_data["nn_config"]["input_size"] = f"{image_size}x{image_size}"
  json_data["nn_config"]["NN_specific_metadata"]["classes"] = number_classes
  json_data["nn_config"]["NN_specific_metadata"]["anchors"] = []
  json_data["nn_config"]["NN_specific_metadata"]["anchor_masks"] = {}
  json_data["nn_config"]["NN_specific_metadata"]["iou_threshold"] = iou_threshold
  json_data["nn_config"]["NN_specific_metadata"]["confidence_threshold"] = confidence_threshold
  json_data["mappings"]["labels"] = labels

with open(f"/content/{json_name}.json", "w") as json_file:
  json.dump(json_data, json_file, indent = 4)

files.download(f"/content/{json_name}.json")

# Model deployment

That's it! You trained your own [YOLOv6](https://github.com/meituan/YOLOv6) object detection model with your custom dataset and converted it to .blob format which is necessary to run inference on the [Luxonis OAK devices](https://docs.luxonis.com/projects/hardware/en/latest/).

> To deploy the YOLOv6 model on your OAK device you can check out the Luxonis GitHub repository for [on-device decoding](https://github.com/luxonis/depthai-experiments/tree/master/gen2-yolo/device-decoding) or use the deployment options from the [**Insect Detect Docs**](https://maxsitt.github.io/insect-detect-docs/software/programming/) (e.g. for continuous automated insect monitoring with the DIY camera trap).